In [1]:
##imports
import numpy as np
import pandas as pd
import spacy

In [5]:
##function to read files and return string
def read_file(filepath):
    
    with open(filepath,  encoding='utf8') as file:
        string = file.read()
    
    return string

In [8]:
##reading .txt file
string = read_file('psychology_of_money.txt') ##use entire filepath if file not in same directory

In [10]:
##loading the language model
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner'])

C:\Users\abhin\anaconda3\envs\nlp_course\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [39]:
##function to remove unnecessary punctuations
def filter_punc(doc):
    return [token.text.lower() for token in nlp(doc) if token.text not in '\n\x0c \n\x0c\x0c \x0c\x0c\x0c               \n\n \n\n   \n\n\n\n \n\n\n\n\n \x0c\x0c\x0c \x0c \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [40]:
##getting tokens
tokens = filter_punc(string)

In [44]:
##generating sequences from the text

train_len = 15 + 1 ##take 15 words as input to generate the 16th word

text_sequences = []

for i in range(train_len, len(tokens)):
    sequence = ' '.join(tokens[i-train_len:i]) ##generate train_len worded text sequence
    text_sequences.append(sequence)
    
len(text_sequences) ##len(tokens) - train_len

55620

In [46]:
##check first three sequences

In [47]:
text_sequences[0]

'for my parents who teach me gretchen who guides me miles and reese who inspire me'

In [48]:
text_sequences[1]

'my parents who teach me gretchen who guides me miles and reese who inspire me introduction'

In [49]:
text_sequences[2]

'parents who teach me gretchen who guides me miles and reese who inspire me introduction the'

In [50]:
##TOKENIZATION

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer() ##instantiation
tokenizer.fit_on_texts(text_sequences)

In [61]:
##function to tokenize each string
def tokenize(string, tokenizer, train_len):
    
    encoded_sequence = tokenizer.texts_to_sequences([string])[0]
    padded_sequence = pad_sequences([encoded_sequence], maxlen=train_len, padding='pre', truncating='pre')[0]
    
    return padded_sequence

In [63]:
##getting encoded sequences
sequences = np.array([tokenize(string=sequence, tokenizer=tokenizer, train_len=train_len) for sequence in text_sequences])

In [66]:
sequences

array([[  12,   79,  516, ...,   56, 6187,  111],
       [  79,  516,   56, ..., 6187,  111, 2271],
       [ 516,   56, 1061, ...,  111, 2271,    1],
       ...,
       [   6,   39,  182, ...,   41,    1, 6189],
       [  39,  182,   34, ...,    1, 6189,    2],
       [ 182,   34,   23, ..., 6189,    2,    1]])

In [67]:
##feature label split

from keras.utils import to_categorical

X, y = sequences[:,:-1], sequences[:,-1]

In [68]:
X

array([[  12,   79,  516, ..., 6188,   56, 6187],
       [  79,  516,   56, ...,   56, 6187,  111],
       [ 516,   56, 1061, ..., 6187,  111, 2271],
       ...,
       [   6,   39,  182, ...,   36,   41,    1],
       [  39,  182,   34, ...,   41,    1, 6189],
       [ 182,   34,   23, ...,    1, 6189,    2]])

In [72]:
seq_len = X.shape[1] ##input length to the neural network

In [76]:
vocab_len = len(tokenizer.index_word) ##length of tokenizer vocabulary

In [77]:
y = to_categorical(y, num_classes=vocab_len+1) ##keras needs one additional column for padding
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [78]:
##MODEL BUILDING
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

In [92]:
##function to create model
def create_model(vocab, seq_len):
    
    model = Sequential()
    
    model.add(Embedding(vocab, 15, input_length=seq_len))
    
    model.add(LSTM(units=10*seq_len, return_sequences=True))
    model.add(LSTM(units=10*seq_len))
    model.add(Dense(units=10*seq_len, activation='relu'))
    
    model.add(Dense(units=vocab, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    model.summary()
    
    return model

In [80]:
##importing dump and load to save model and tokenizer
from pickle import dump, load

dump(tokenizer, open('tokenizer','wb')) ##save the tokenizer

In [93]:
##create model
model = create_model(vocab=vocab_len+1, seq_len=seq_len) ##keras needs one extra column for padding

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 15, 15)            92850     
_________________________________________________________________
lstm_7 (LSTM)                (None, 15, 150)           99600     
_________________________________________________________________
lstm_8 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_7 (Dense)              (None, 150)               22650     
_________________________________________________________________
dense_8 (Dense)              (None, 6190)              934690    
Total params: 1,330,390
Trainable params: 1,330,390
Non-trainable params: 0
_________________________________________________________________


In [94]:
##fit the model
model.fit(X, y, batch_size=128, epochs=300)

Epoch 1/300
55620/55620 [==============================] - 61s 1ms/step - loss: 6.9283 - accuracy: 0.0453
Epoch 2/300
55620/55620 [==============================] - 56s 1ms/step - loss: 6.5958 - accuracy: 0.0474
Epoch 3/300
55620/55620 [==============================] - 50s 893us/step - loss: 6.4207 - accuracy: 0.0585
Epoch 4/300
55620/55620 [==============================] - 49s 887us/step - loss: 6.2805 - accuracy: 0.0651
Epoch 5/300
55620/55620 [==============================] - 49s 885us/step - loss: 6.1715 - accuracy: 0.0700
Epoch 6/300
55620/55620 [==============================] - 50s 897us/step - loss: 6.0610 - accuracy: 0.0767
Epoch 7/300
55620/55620 [==============================] - 54s 962us/step - loss: 6.0224 - accuracy: 0.0780
Epoch 8/300
55620/55620 [==============================] - 59s 1ms/step - loss: 5.8718 - accuracy: 0.0929
Epoch 9/300
55620/55620 [==============================] - 59s 1ms/step - loss: 5.7579 - accuracy: 0.1007
Epoch 10/300
55620/55620 [==========

55620/55620 [==============================] - 47s 849us/step - loss: 2.5065 - accuracy: 0.4153
Epoch 77/300
55620/55620 [==============================] - 48s 858us/step - loss: 2.4787 - accuracy: 0.4216
Epoch 78/300
55620/55620 [==============================] - 47s 847us/step - loss: 2.4614 - accuracy: 0.4233
Epoch 79/300
55620/55620 [==============================] - 47s 848us/step - loss: 2.4281 - accuracy: 0.4314
Epoch 80/300
55620/55620 [==============================] - 47s 848us/step - loss: 2.4071 - accuracy: 0.4371
Epoch 81/300
55620/55620 [==============================] - ETA: 0s - loss: 2.3819 - accuracy: 0.43 - 47s 853us/step - loss: 2.3822 - accuracy: 0.4392
Epoch 82/300
55620/55620 [==============================] - 47s 852us/step - loss: 2.3581 - accuracy: 0.4439
Epoch 83/300
55620/55620 [==============================] - 47s 851us/step - loss: 2.3350 - accuracy: 0.4469
Epoch 84/300
55620/55620 [==============================] - 47s 854us/step - loss: 2.3132 - accurac

55620/55620 [==============================] - 47s 843us/step - loss: 1.0999 - accuracy: 0.7188
Epoch 151/300
55620/55620 [==============================] - 47s 840us/step - loss: 1.0855 - accuracy: 0.7219
Epoch 152/300
55620/55620 [==============================] - 47s 842us/step - loss: 1.0894 - accuracy: 0.7201
Epoch 153/300
55620/55620 [==============================] - 47s 842us/step - loss: 1.0688 - accuracy: 0.7255
Epoch 154/300
55620/55620 [==============================] - 47s 840us/step - loss: 1.0589 - accuracy: 0.7294
Epoch 155/300
55620/55620 [==============================] - 47s 842us/step - loss: 1.0401 - accuracy: 0.7329
Epoch 156/300
55620/55620 [==============================] - 47s 844us/step - loss: 1.0111 - accuracy: 0.7427
Epoch 157/300
55620/55620 [==============================] - 47s 845us/step - loss: 1.0046 - accuracy: 0.7431
Epoch 158/300
55620/55620 [==============================] - 47s 848us/step - loss: 1.0143 - accuracy: 0.7368
Epoch 159/300
55620/5562

55620/55620 [==============================] - 46s 834us/step - loss: 0.4613 - accuracy: 0.8748
Epoch 225/300
55620/55620 [==============================] - 46s 832us/step - loss: 0.4029 - accuracy: 0.8964
Epoch 226/300
55620/55620 [==============================] - 46s 832us/step - loss: 0.3964 - accuracy: 0.8969
Epoch 227/300
55620/55620 [==============================] - 46s 833us/step - loss: 0.3641 - accuracy: 0.9087
Epoch 228/300
55620/55620 [==============================] - 46s 832us/step - loss: 0.3582 - accuracy: 0.9096
Epoch 229/300
55620/55620 [==============================] - 46s 832us/step - loss: 0.3827 - accuracy: 0.9007
Epoch 230/300
55620/55620 [==============================] - 47s 840us/step - loss: 0.4371 - accuracy: 0.8792
Epoch 231/300
55620/55620 [==============================] - 46s 834us/step - loss: 0.4290 - accuracy: 0.8815
Epoch 232/300
55620/55620 [==============================] - 46s 832us/step - loss: 0.3754 - accuracy: 0.9017
Epoch 233/300
55620/5562

55620/55620 [==============================] - 46s 831us/step - loss: 0.2030 - accuracy: 0.9449
Epoch 299/300
55620/55620 [==============================] - 46s 832us/step - loss: 0.1586 - accuracy: 0.9593
Epoch 300/300
55620/55620 [==============================] - 46s 831us/step - loss: 0.1151 - accuracy: 0.9757


In [95]:
##save the model
model.save('model.h5')

In [97]:
##load saved model and tokenzer
from keras.models import load_model

model = load_model('model.h5')

with open('tokenizer', 'rb') as file:
    tokenizer = load(file)

In [113]:
##generating sequences using model and tokenizer

def generate_sequences(model, tokenizer, seq_len, seed_text, num_words):
    
    output_text = [] ##list to collect output
    
    input_text = seed_text ##take 15 words as user input to initialize text generation
    
    for i in range(num_words):
        
        encoded_input = tokenizer.texts_to_sequences([input_text])[0] ##encode input using tokenizer vocabulary
        
        padded_input = pad_sequences([encoded_input], maxlen=seq_len, truncating='pre', padding='pre') ##pad sequence
        
        word_index = model.predict_classes(padded_input)[0] ##getting the index of the next word in tokenizer.index_word
        
        word = tokenizer.index_word[word_index] ##word from tokenizer vocabulary
        
        input_text += ' ' + word ##add word to next iteration input
        
        output_text.append(word)
    
    return ' '.join(output_text)

In [120]:
##SOME EXAMPLES OF TEXT GENERATION USING OUR MODEL

In [121]:
seed_text = "the most important thing to understand about personal finance it its pertinence in modern human life"
generate_sequences(model=model, tokenizer=tokenizer, seq_len=seq_len, seed_text=seed_text, num_words=20)

'in the aid of imagination in america and if a point of growth and and even eﬀort ⁵ and wildly'

In [123]:
seed_text = "the reason why many people stay poor for a majority of their years is because they are yet to understand"
generate_sequences(model=model, tokenizer=tokenizer, seq_len=seq_len, seed_text=seed_text, num_words=100)

'our tendency and their goals i sleep out a day what people book did n’t be rational at the number of my own head i expect more the ones on the power of reasonable going this do you might have mostly incomes ” as i wiped out paying not to to be understood is attributable to those who things are found the opposite on productivity now you are in the way anyone expected you do n’t need to expect them better people crime wealth their decisions is so taking that magic in a mountaineering chapter as much progress buﬀett else'

In [124]:
seed_text ='no seed text'
generate_sequences(model=model, tokenizer=tokenizer, seq_len=seq_len, seed_text=seed_text, num_words=10)

'initial some opportunities were the best price available to be'